## Final Project Submission

Please fill out:
* Student name: Dhruv Ragunathan
* Student pace: part time
* Scheduled project review date/time: 
* Instructor name: 
* Blog post URL:


In [ ]:
# Your code here - remember to use markdown cells for comments as well!

# Introduction and Business Case

The purpose of this notebook is too analyze the housing data in King's County to provide Blackmineral (any similarity to real world companies are coincidential) information on purchasing property in that region.

The analysis in this notebook consists of analyzing KC Housing Data and creating a model to help predict sale price based on certain variables.

Managing Directors at Blackmineral will use these models to determine their strategy for purchasing property in King's County.

In [2]:
import pandas as pd
import numpy as np

In [73]:
kc_house_df = pd.read_csv('data/kc_house_data.csv')

kc_house_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21597 non-null  int64  
 1   date           21597 non-null  object 
 2   price          21597 non-null  float64
 3   bedrooms       21597 non-null  int64  
 4   bathrooms      21597 non-null  float64
 5   sqft_living    21597 non-null  int64  
 6   sqft_lot       21597 non-null  int64  
 7   floors         21597 non-null  float64
 8   waterfront     19221 non-null  object 
 9   view           21534 non-null  object 
 10  condition      21597 non-null  object 
 11  grade          21597 non-null  object 
 12  sqft_above     21597 non-null  int64  
 13  sqft_basement  21597 non-null  object 
 14  yr_built       21597 non-null  int64  
 15  yr_renovated   17755 non-null  float64
 16  zipcode        21597 non-null  int64  
 17  lat            21597 non-null  float64
 18  long  

# Data Understanding

* Data description: The csv contains house sales data in King's county. This data contains records of houses sold in King's county with variables such as the price, bedrooms, bathrooms, view etc.

* Data use: This dataset can be used to create models that help predict prices of houses in King County.

* Size of Data: The dataset has 21597 records and 21 columns.

* Analysis method: Multi-linear regression will be used to create models that show the impact of variables on price.

* Data Limitations: This data set is limited by the amount of variables recorded during the house sale. There are other key points to consider when evaluating a potential sale price such as quality of the school districts, noise in the area, etc.

# Data Preparation

First review the data and find and clean missing values

In [9]:
kc_house_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21597 non-null  int64  
 1   date           21597 non-null  object 
 2   price          21597 non-null  float64
 3   bedrooms       21597 non-null  int64  
 4   bathrooms      21597 non-null  float64
 5   sqft_living    21597 non-null  int64  
 6   sqft_lot       21597 non-null  int64  
 7   floors         21597 non-null  float64
 8   waterfront     19221 non-null  object 
 9   view           21534 non-null  object 
 10  condition      21597 non-null  object 
 11  grade          21597 non-null  object 
 12  sqft_above     21597 non-null  int64  
 13  sqft_basement  21597 non-null  object 
 14  yr_built       21597 non-null  int64  
 15  yr_renovated   17755 non-null  float64
 16  zipcode        21597 non-null  int64  
 17  lat            21597 non-null  float64
 18  long  

In [11]:
# Determine where there are missing values

kc_house_df.isna().sum()

id                  0
date                0
price               0
bedrooms            0
bathrooms           0
sqft_living         0
sqft_lot            0
floors              0
waterfront       2376
view               63
condition           0
grade               0
sqft_above          0
sqft_basement       0
yr_built            0
yr_renovated     3842
zipcode             0
lat                 0
long                0
sqft_living15       0
sqft_lot15          0
dtype: int64

In [13]:
#Determine percentage of records missing in each category

num_records = len(kc_house_df)

prc_wf_msg = round(kc_house_df.isna().sum()['waterfront']/num_records * 100,1)

prc_yr_ren_msg = round(kc_house_df.isna().sum()['yr_renovated']/num_records*100,1)

prc_view_msg = round(kc_house_df.isna().sum()['view']/num_records*100,1)


print("The percent of records missing for waterfront are ",prc_wf_msg)

print("The percent of records missing for view are ",prc_view_msg)

print("The percent of records missing for renovated year are ",prc_yr_ren_msg)


The percent of records missing for waterfront are  11.0
The percent of records missing for view are  0.3
The percent of records missing for renovated year are  17.8


The number of records missing a view are neglible (0.3%), so all those rows can be dropped.

In [34]:
# Drop all rows missing the view

kc_house_df = kc_house_df.loc[kc_house_df['view'].dropna().index]

kc_house_df.isna().sum() # all Records where view is NA are dropped

In [37]:
# The column renovated year will be dropped in the data frame since nearly 20% of the data is missing

kc_house_df = kc_house_df.drop(columns = 'yr_renovated')

kc_house_df

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,NaN,NONE,Average,7 Average,1180,0.0,1955,98178,47.5112,-122.257,1340,5650
1,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,NO,NONE,Average,7 Average,2170,400.0,1951,98125,47.7210,-122.319,1690,7639
2,5631500400,2/25/2015,180000.0,2,1.00,770,10000,1.0,NO,NONE,Average,6 Low Average,770,0.0,1933,98028,47.7379,-122.233,2720,8062
3,2487200875,12/9/2014,604000.0,4,3.00,1960,5000,1.0,NO,NONE,Very Good,7 Average,1050,910.0,1965,98136,47.5208,-122.393,1360,5000
4,1954400510,2/18/2015,510000.0,3,2.00,1680,8080,1.0,NO,NONE,Average,8 Good,1680,0.0,1987,98074,47.6168,-122.045,1800,7503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21592,263000018,5/21/2014,360000.0,3,2.50,1530,1131,3.0,NO,NONE,Average,8 Good,1530,0.0,2009,98103,47.6993,-122.346,1530,1509
21593,6600060120,2/23/2015,400000.0,4,2.50,2310,5813,2.0,NO,NONE,Average,8 Good,2310,0.0,2014,98146,47.5107,-122.362,1830,7200
21594,1523300141,6/23/2014,402101.0,2,0.75,1020,1350,2.0,NO,NONE,Average,7 Average,1020,0.0,2009,98144,47.5944,-122.299,1020,2007
21595,291310100,1/16/2015,400000.0,3,2.50,1600,2388,2.0,NaN,NONE,Average,8 Good,1600,0.0,2004,98027,47.5345,-122.069,1410,1287


In [38]:
# Get breakdown of yes versus no

kc_house_df["waterfront"].value_counts()

NO     19019
YES      145
Name: waterfront, dtype: int64

Finally for Waterfront, we cannot drop columns with missing data since that's 10% of the overall dataset.

I would be interested to see the impact of a house being near a waterfront on the price.

Since most of the filled data is overwhelmingly "NO", I will fill the missing values with No since this won't impact the dataset significantly.

In [41]:
# Fill missing data for waterfront with NO

kc_house_df = kc_house_df.fillna(value= "NO")

kc_house_df["waterfront"].value_counts() # Confirm number of "NO" has increased

NO     21389
YES      145
Name: waterfront, dtype: int64

Note that the proportion of NO has increased from 99.2% to 99.3%. This would have a neglible effect on the model.

In [46]:
# Based on the info of the dataframe, the following columns are numeric or categorical
# Numeric: date, rice, bedrooms, bathrooms, sqft_living, sqft_lot, floors, sqft_above, yr_built, lat, long, sqft_living15, sqft_lot15
# Numeric Category: zipcode
# String Category: waterfront, view, condition, grade

kc_house_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21534 entries, 0 to 21596
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21534 non-null  int64  
 1   date           21534 non-null  object 
 2   price          21534 non-null  float64
 3   bedrooms       21534 non-null  int64  
 4   bathrooms      21534 non-null  float64
 5   sqft_living    21534 non-null  int64  
 6   sqft_lot       21534 non-null  int64  
 7   floors         21534 non-null  float64
 8   waterfront     21534 non-null  object 
 9   view           21534 non-null  object 
 10  condition      21534 non-null  object 
 11  grade          21534 non-null  object 
 12  sqft_above     21534 non-null  int64  
 13  sqft_basement  21534 non-null  object 
 14  yr_built       21534 non-null  int64  
 15  zipcode        21534 non-null  int64  
 16  lat            21534 non-null  float64
 17  long           21534 non-null  float64
 18  sqft_l

In [52]:
# Confirm that categorical/string variables are formatted correctly in dataset

kc_house_df["view"].value_counts()

kc_house_df["condition"].value_counts()

kc_house_df["grade"].value_counts()

kc_house_df["sqft_basement"].value_counts()




0.0       12798
?           452
600.0       216
500.0       209
700.0       207
          ...  
1245.0        1
65.0          1
1281.0        1
225.0         1
248.0         1
Name: sqft_basement, Length: 302, dtype: int64

### Data Cleaning Numeric Categories

The sqft_basement column is an object in the dataframe even though it should be numeric. This is because values in that field are "?" a string. The strategy to clean this would be to replace them with the average basement square footage. 452 records are not significant, but would prefer not to drop the data.

In [65]:
# Calculate the mean sqft_basement. Replace "?" with the mean

sqft_basement_num = kc_house_df[kc_house_df["sqft_basement"] != "?"]["sqft_basement"]

sqft_basement_num = sqft_basement_num.astype('float64')

avg_sqft_basement = round(sqft_basement_num.mean(),1) # data is rounded to one-decimal place in dataframe
 
avg_sqft_basement

291.4

In [67]:
# Replace the ? to the avg square basement size

kc_house_df["sqft_basement"].replace(to_replace="?",value = avg_sqft_basement,inplace = True)

In [72]:
# Convert datatype from object to float now

kc_house_df["sqft_basement"] = kc_house_df["sqft_basement"].astype('float64')

kc_house_df.info() # 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21534 entries, 0 to 21596
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21534 non-null  int64  
 1   date           21534 non-null  object 
 2   price          21534 non-null  float64
 3   bedrooms       21534 non-null  int64  
 4   bathrooms      21534 non-null  float64
 5   sqft_living    21534 non-null  int64  
 6   sqft_lot       21534 non-null  int64  
 7   floors         21534 non-null  float64
 8   waterfront     21534 non-null  object 
 9   view           21534 non-null  object 
 10  condition      21534 non-null  object 
 11  grade          21534 non-null  object 
 12  sqft_above     21534 non-null  int64  
 13  sqft_basement  21534 non-null  float64
 14  yr_built       21534 non-null  int64  
 15  zipcode        21534 non-null  int64  
 16  lat            21534 non-null  float64
 17  long           21534 non-null  float64
 18  sqft_l

In [44]:
kc_house_df.describe()

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,sqft_above,yr_built,zipcode,lat,long,sqft_living15,sqft_lot15
count,2.153400e+04,2.153400e+04,21534.000000,21534.000000,21534.000000,2.153400e+04,21534.000000,21534.000000,21534.000000,21534.000000,21534.000000,21534.000000,21534.000000,21534.000000
mean,4.582351e+09,5.400577e+05,3.373038,2.115712,2079.827854,1.509060e+04,1.494126,1788.557537,1971.002275,98077.939352,47.560180,-122.213948,1986.299944,12751.079502
std,2.876779e+09,3.660596e+05,0.926410,0.768602,917.446520,4.138021e+04,0.539806,827.745641,29.376044,53.506639,0.138528,0.140735,685.121001,27255.483308
min,1.000102e+06,7.800000e+04,1.000000,0.500000,370.000000,5.200000e+02,1.000000,370.000000,1900.000000,98001.000000,47.155900,-122.519000,399.000000,651.000000
25%,2.123212e+09,3.220000e+05,3.000000,1.750000,1430.000000,5.040000e+03,1.000000,1190.000000,1951.000000,98033.000000,47.471200,-122.328000,1490.000000,5100.000000
50%,3.904945e+09,4.500000e+05,3.000000,2.250000,1910.000000,7.617000e+03,1.500000,1560.000000,1975.000000,98065.000000,47.571900,-122.230000,1840.000000,7620.000000
75%,7.312175e+09,6.450000e+05,4.000000,2.500000,2550.000000,1.068775e+04,2.000000,2210.000000,1997.000000,98118.000000,47.678100,-122.125000,2360.000000,10083.000000
max,9.900000e+09,7.700000e+06,33.000000,8.000000,13540.000000,1.651359e+06,3.500000,9410.000000,2015.000000,98199.000000,47.777600,-121.315000,6210.000000,871200.000000


# Modeling

First let's create a baseline model.

# Regression Results

Analysis of Regression results

# Conclusion and Recommendations
